Face Detection Using OpenCV

In [ ]:
from IPython import get_ipython
get_ipython().run_line_magic('run', 'main.ipynb') 
model = EmotionCNN()
model.load_state_dict(torch.load('emotion_recognition_model.pth'))
model.eval()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(128 * 6 * 6, 128) 
        self.fc2 = nn.Linear(128, 8) 
        
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = self.pool3(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 6 * 6) 
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x



In [ ]:
import cv2
import numpy as np
import torch
from IPython import get_ipython

get_ipython().run_line_magic('run', 'main.ipynb')  

model = EmotionCNN()
model.load_state_dict(torch.load('emotion_recognition_model.pth'))
model.eval()

emotions = ['angry','contempt' 'disgust', 'fear', 'neutral', 'happiness', 'sadness', 'surprise']

img = cv2.imread('/home/shobhnikk/Documents/MusicProject/tryimage7.png')

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

if len(faces) > 0:
    x, y, w, h = faces[0]
    face = img[y:y+h, x:x+w]  

    face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face_resized = cv2.resize(face_gray, (48, 48))

    face_resized = face_resized.astype('float32') / 255  
    face_resized = np.expand_dims(face_resized, axis=-1)  
    face_resized = np.expand_dims(face_resized, axis=0)  

    face_resized = torch.tensor(face_resized)  

    face_resized = face_resized.squeeze(-1) 
    face_resized = face_resized.unsqueeze(1) 

    with torch.no_grad():  
        emotion_prob = model(face_resized)
        emotion = emotions[torch.argmax(emotion_prob)]  

    print(f"Predicted Emotion: {emotion}")

    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    cv2.imshow('Emotion Detection', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No face detected.")


In [ ]:
import pandas as pd
import torch
from PIL import Image
from torchvision import transforms

csv_path = "change it with the path of the csv file"
data = pd.read_csv(csv_path)

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),  
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))
])

images = []
labels = []
for idx, row in data.iterrows():
    image_path = row['image_path']
    label = row['label'] 
    
    image = Image.open(image_path)
    image = transform(image)
    images.append(image)
    labels.append(label)

images = torch.stack(images)
if labels:  
    labels = torch.tensor(labels)
